# ODE solver performance battle

The point of this jupyter notebook is to look at how some of the available ODE solvers perform for our particular transmon simulation. Up until now, we used the QuTip python library, which abstract away the use of these solvers. This makes the code simpler and easy to read, but hides the heart of the problem, which in essence is to numerically solve a differential equation. For our system, this is the Schrödinger equation:
$$
i\hbar\frac{\partial}{\partial t}\ket{\psi} - \hat{H(t)}\ket{\psi}
$$
This can be solved with the usual tools, namely any type of Runge-Kutta method. QuTip has a few options under the hood for time-dependent Hamiltonians, they are all adaptive solvers. Meaning that they estimate the **local truncation error (LTE)**—the error introduced in a single step—and compare it to the user's requested **absolute tolerance (atol)** and **relative tolerance (rtol)**.

### 1. Embedded Runge-Kutta Methods
* **Methods:** `'dop853'`, `'tsit5'`, `'vern7'`, `'vern9'`

These methods use an "embedded pair" or "FSAL" (First Same As Last) strategy.

* **How it works:** In a single step, the algorithm uses its set of coefficients (the Butcher tableau) to compute two different solutions:
    1.  A high-order solution: $y_{n+1}$ (e.g., 8th-order for `'dop853'`)
    2.  A lower-order solution: $\hat{y}_{n+1}$ (e.g., 5th-order for `'dop853'`)

* **Error Estimation:** The difference between these two solutions, $E = ||y_{n+1} - \hat{y}_{n+1}||$, provides a highly efficient and accurate estimate of the local error.

* **Step Control:**
    1.  **If $E > \text{Tolerance}$:** The estimated error is too high. The step is **rejected**. The step size $h$ is significantly reduced (e.g., $h_{\text{new}} < h_{\text{old}}$), and the step is re-computed with the new, smaller $h$.
    2.  **If $E \le \text{Tolerance}$:** The error is acceptable. The step is **accepted**, and the high-order solution $y_{n+1}$ is used. The solver then calculates an optimal $h_{\text{new}}$ for the *next* step, often increasing it slightly to improve efficiency, based on the formula $h_{\text{new}} = h_{\text{old}} \times (\text{Tolerance} / E)^{1/p}$, where $p$ is the order of the error estimator.

### 2. Linear Multistep Methods
* **Methods:** `'adams'`, `'bdf'`

These methods use a **predictor-corrector** mechanism to estimate the error.

* **How it works:**
    1.  **Predict:** An *explicit* formula (e.g., Adams-Bashforth) uses previous steps ($y_n, y_{n-1}, \dots$) to predict a "guess" for the next step, $y_{n+1}^{(p)}$.
    2.  **Correct:** An *implicit* formula (e.g., Adams-Moulton) uses the predicted value $y_{n+1}^{(p)}$ to solve for a more accurate, final value, $y_{n+1}^{(c)}$.

* **Error Estimation:** The difference between the predicted guess and the final corrected value, $E = ||y_{n+1}^{(c)} - y_{n+1}^{(p)}||$, serves as a good estimate of the local error.

* **Step Control:**
    * The logic is the same as for Runge-Kutta: if $E$ is too large, the step is rejected and $h$ is reduced. If $E$ is acceptable, the step is taken, and $h$ is adjusted for the next step.
    * **Adaptive Order:** A key feature of these methods is that they are also **adaptive-order**. If the error is consistently very low, they may *increase their order* (e.g., from a 3-step to a 4-step BDF) to take larger, more efficient steps. If the error is hard to control, they may decrease their order.

### 3. Special Cases
* **`'lsoda'`**
    This solver **does not have its own step-size logic**. It is a "meta-solver" that simply uses the step-size control logic of whichever method it is currently running:
    * If the problem is non-stiff, it runs `'adams'` and uses its predictor-corrector error estimation.
    * If it detects stiffness, it switches to `'bdf'` and uses its predictor-corrector error estimation.

* **`'diag'`**
    This method has **no step-size control**. It is not an integrator. It calculates the exact analytical solution $y_k(t) = y_k(0) e^{\lambda_k t}$ for every time $t$ specified in the user's `tlist`. The "steps" are just the intervals in the `tlist` you provide.

* **`'krylov'`**
    This method's error control is **not based on the time step $h$**.
    * Its accuracy is controlled by the **dimension ($m$) of the Krylov subspace**. The error is the difference between the true matrix exponential $e^{Lh}y$ and its approximation $V_m e^{H_m h} V_m^T y$.
    * The time step $h$ itself is **fixed**. It is either the interval between points in your `tlist` ($\Delta t = t_{n+1} - t_n$) or, if you set `options.nsteps`, a smaller, fixed internal step ($h = \Delta t / \text{nsteps}$). It does not adapt $h$ based on a temporal error estimate.

In [160]:
import numpy as np
import time
import os
from scipy.sparse import (
    identity as sparse_identity,
    diags as sparse_diags,
    kron as sparse_kron,
    csc_matrix,
    eye as sparse_eye
)
from scipy.sparse.linalg import expm as sparse_expm
from scipy.integrate import solve_ivp, RK45
from scipy.integrate._ivp.base import OdeSolver
from functools import reduce # Needed for sparse.kron reduction
import sys
from pathlib import Path
# Add the project root directory to Python path
notebook_path = Path().absolute()  # Gets the current notebook directory
project_root = notebook_path.parent  # Goes up one level to project root
sys.path.append(str(project_root))
from utils.funcs import load_params
import matplotlib.pyplot as plt

### Operator functions

In [161]:
# --- SciPy/NumPy Operator Definitions ---

def sparse_destroy(dim):
    """Creates a sparse destruction operator."""
    if dim <= 0:
        raise ValueError("Dimension must be > 0")
    if dim == 1:
        return csc_matrix((1, 1), dtype=np.complex128)
    data = np.sqrt(np.arange(1, dim, dtype=np.complex128))
    offsets = [1]
    # FIX: Wrap 'data' in a list to match the list 'offsets'
    return sparse_diags([data], offsets, shape=(dim, dim), format='csc')

def sparse_num(dim):
    """Creates a sparse number operator."""
    if dim <= 0:
        raise ValueError("Dimension must be > 0")
    if dim == 1:
        return csc_matrix((1, 1), dtype=np.complex128)
    data = np.arange(dim, dtype=np.complex128)
    offsets = [0]
    # FIX: Wrap 'data' in a list to match the list 'offsets'
    return sparse_diags([data], offsets, shape=(dim, dim), format='csc')

def get_sparse_op(op, site, dims):
    """
    Creates a full-system sparse operator from a local operator
    using sparse.kron.
    """
    op_list = [sparse_identity(d, dtype=np.complex128, format='csc') for d in dims]
    op_list[site] = op
    return reduce(sparse_kron, op_list)

def propagator_ode_real(t, y_real, H0, H1, W, nu_delta, D):
    """
    The ODE function for the propagator U(t), compatible with solve_ivp.
    It evolves dU/dt = -i * H(t) * U(t)
    
    y_real = [U_flat_real, U_flat_imag] (size 2 * D*D)
    """
    D_sq = D * D
    # Reconstruct the complex flattened vector
    U_flat_complex = y_real[:D_sq] + 1j * y_real[D_sq:]
    
    # Reshape into a D x D matrix
    # We use 'F' (Fortran) order to match the flattening,
    # which is standard for quantum mechanics state vectors vs operators.
    U = U_flat_complex.reshape((D, D), order='F')
    
    # Calculate H(t)
    H_t = H0 + H1 * (W * np.cos(nu_delta * t))
    
    # Calculate dU/dt = -i * H(t) * U
    # H_t is (D, D), U is (D, D). Use matrix multiplication
    dU_dt_complex = -1j * (H_t @ U)
    
    # Flatten the complex derivative
    dU_dt_flat = dU_dt_complex.flatten(order='F')
    
    # Split back into real and imaginary parts
    return np.concatenate([dU_dt_flat.real, dU_dt_flat.imag])

### Build Hamiltonian

Construct the time-independent (H0) and time-dependent (H1) Hamiltonian parts as SciPy sparse matrices.

In [162]:

# --- Evaluate parameters ---
params = load_params('example.json')

N = int(params['N'])
n_max_transmon = int(params['n_max_transmon'])
n_max_resonator = int(params['n_max_resonator'])

eta = float(params['eta'])
phiq = float(params['phiq'])
phia = eval(str(params['phia']))
J = eval(str(params['J']))
nu = eval(str(params['nu']))
delta = eval(str(params['delta']))
de = float(params['de']) * delta
wq = eval(str(params['wq']))
EJ = eval(str(params['EJ']))

W = eta * (wq**2 - (nu - delta)**2) / (2 * wq)
nu_delta = nu - delta
period = 2 * np.pi / nu_delta

# --- System Definition ---
dims = [n_max_transmon if i % 2 == 0 else n_max_resonator for i in range(2 * N)]
D_total = np.prod(dims)

En = []
for i in range(N):
    En.append(wq + i * de)
    En.append(nu)
En = np.array(En)

# --- Local Operators (Sparse) ---

a_t = sparse_destroy(n_max_transmon)
a_t_dag = a_t.conj().T.tocsc() 
n_t = sparse_num(n_max_transmon)
x_t = a_t_dag + a_t           

a_r = sparse_destroy(n_max_resonator)
a_r_dag = a_r.conj().T.tocsc() 
n_r = sparse_num(n_max_resonator)
x_r = a_r_dag + a_r           

print(f"Total Hilbert space dimension: {D_total}")

# --- Build Hamiltonian (Sparse) ---
print("Building sparse H0...")
# On-site energies
H0 = csc_matrix((D_total, D_total), dtype=np.complex128)
for i in range(2 * N):
    if i % 2 == 0:  # Transmon site
        H0 += En[i] * get_sparse_op(n_t, i, dims)
    else:  # Resonator site
        H0 += En[i] * get_sparse_op(n_r, i, dims)

# EJ term: -0.5*EJ*(phi^2 + cos(phi))
phi_squared_sum = csc_matrix((D_total, D_total), dtype=np.complex128)
cos_phi_sum = csc_matrix((D_total, D_total), dtype=np.complex128)

for i in range(N):
    print(f"Building phi term for site {i}...")
    # phi_op for the i-th Transmon-Resonator pair
    phi_op = (
        phiq * get_sparse_op(x_t, 2 * i, dims) + 
        phia * get_sparse_op(x_r, 2 * i + 1, dims)
    )
    
    # Contribution to the sum of phi^2 terms
    phi_squared_sum += phi_op * phi_op
    
    # Contribution to the sum of cos(phi) terms
    # This is the computationally heavy step
    print(f"Calculating matrix exponential for site {i}...")
    U_phi = sparse_expm(-1j * phi_op)
    cos_phi_sum += 0.5 * (U_phi + U_phi.conj().T)
    print(f"Done with site {i}.")

H0 += EJ * (0.5 * phi_squared_sum + cos_phi_sum)

# J coupling term
for i in range(N - 1):
    H0 += J * get_sparse_op(x_t, 2 * i, dims) * get_sparse_op(x_t, 2 * i + 2, dims)

print("Building sparse H1...")
# Time-dependent part of Hamiltonian
H1 = csc_matrix((D_total, D_total), dtype=np.complex128)
for i in range(N):
    H1 += get_sparse_op(x_t, 2 * i, dims)

# --- Build Initial State (NumPy) ---
psi_list = []
for i in range(2 * N):
    dim_i = dims[i] # Get the correct dimension for the site
    state_vec = np.zeros(dim_i, dtype=np.complex128)
    if i == 0:
        state_vec[1] = 1.0 # Transmon 1 is |1>
    else:
        state_vec[0] = 1.0 # All others are |0>
    psi_list.append(state_vec)

# Use reduce with np.kron for the state vector
psi0_vec = reduce(np.kron, psi_list)

# --- Build Observables (Sparse) ---
number_ops = []
for i in range(2 * N):
    if i % 2 == 0:
        number_ops.append(get_sparse_op(n_t, i, dims))
    else:
        number_ops.append(get_sparse_op(n_r, i, dims))
        


Total Hilbert space dimension: 256
Building sparse H0...
Building phi term for site 0...
Calculating matrix exponential for site 0...
Done with site 0.
Building phi term for site 1...
Calculating matrix exponential for site 1...
Done with site 1.
Building sparse H1...


### Monkey patch scipy's custom RK4 solver

In [163]:
from scipy.integrate._ivp.base import OdeSolver

class CustomRK4(OdeSolver):
    """
    A custom Runge-Kutta 4 solver with a fixed step size (h), implemented 
    as a subclass of SciPy's internal OdeSolver base class.
    
    This solver is selected by setting 'solver_method': 'CUSTOM_RK4'.
    The fixed step size h is derived from the 'usteps' parameter.
    """
    
    order = 4 # Order of the integration method
    
    def __init__(self, fun, t0, y0, t_bound, vectorized, **extraneous):
        # Initialize base class (handles fun, t0, y0, t_bound, vectorized)
        h = extraneous.pop('max_step', np.inf)         
        # 'max_step' passed by solve_ivp will be used here to convey the fixed step size 'h'
        if h == np.inf:
            raise ValueError("CustomRK4 requires a finite step size (h). Check 'usteps' calculation.")
        
        super().__init__(fun, t0, y0, t_bound, vectorized, **extraneous)
        self.h = h
        
    def _step_impl(self):
        """Performs one fixed-size RK4 step."""
        t, y = self.t, self.y
        h = self.h
        
        # Check if the step would go beyond the boundary
        if t + h >= self.t_bound:
            h = self.t_bound - t
            if h <= 1e-15: # Time step is effectively zero
                self.status = 'finished'
                return False, False # Stop integration
        
        # RK4 coefficients calculation
        try:
            k1 = self.fun(t, y)
            k2 = self.fun(t + h / 2, y + h * k1 / 2)
            k3 = self.fun(t + h / 2, y + h * k2 / 2)
            k4 = self.fun(t + h, y + h * k3)
        except Exception as e:
            self.status = 'failed'
            self.message = f"Error during RHS evaluation in CustomRK4: {e}"
            return False, False

        # Update y using RK4 formula
        self.y += h / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
        self.t += h
        
        # Check if we reached the boundary (or very close)
        if self.t >= self.t_bound - 1e-15:
            self.status = 'finished'

        # Success=True, Error_estimate=False (fixed step)
        return True, False 

    def _dense_output_impl(self):
        # Dense output is not implemented or required for this use case.
        return None

### Test a custom RK4 vs Scipy's RK4

Test different solvers and check their performance.

Solvers:
- 'RK45' (Adaptive, high-accuracy)
- 'CUSTOM_RK4' (Slow, pure-Python fixed-step for convergence tests)

In [164]:
# Set up simulation length and steps 

# Read solver configuration
rtol = params.get('rtol', 1e-6)
atol = params.get('atol', 1e-8)

sim_length = int(params['sim_length'])
N = int(params['N'])
D_total = np.prod(dims)
D_sq = D_total * D_total

# Number of steps for one period propagation
usteps = int(params['usteps'])

# --- Set up Time List for one period ---
tlist_one_period = np.linspace(0, period, usteps)

# Calculate the step size used for evaluation points
U_dt = tlist_one_period[1] - tlist_one_period[0]
params['U_dt'] = U_dt

# --- Set up stroboscopic time list for full evolution ---
tUsteps = int(sim_length / period)
tlist = np.linspace(0, tUsteps * period, tUsteps + 1)
params['sim_dt'] = period

print(f"Drive period: {period:.4f} ns")


Drive period: 0.1394 ns


### 1. Scipy's RK45

In [165]:

# --- Prepare for ODE Solver (to find U) ---
# Initial state is the flattened identity matrix
I_flat = sparse_eye(D_total, D_total, dtype=np.complex128, format='csc').toarray().flatten(order='F')
y0_real = np.concatenate([I_flat.real, I_flat.imag])

# ODE arguments
ode_args = (H0, H1, W, nu_delta, D_total)

# This takes the expected (t, y) and passes them to your function *with* the extra args
fun_wrapped = lambda t, y: propagator_ode_real(t, y, *ode_args)

# Dictionary to store results
solver_results = {}

# ============================================
# Test 1: SciPy's RK45
# ============================================
print("\n" + "="*60)
print("TEST 1: SciPy's Default RK45 Solver (Adaptive)")
print("="*60)

start_time = time.time()
print(f"Starting RK45 solver...")
sol_U_rk45 = RK45(
    fun_wrapped,
    t0 = 0,
    y0=y0_real,
    t_bound = period,
    first_step=None,
    max_step = U_dt,
    rtol=rtol,
    atol=atol,
    vectorised=True,
)
rk45_time = time.time() - start_time
# diffs = np.diff(sol_U_rk45.t, axis=0)
# diff_config = np.diff(tlist_one_period, axis=0)
# print(dir(sol_U_rk45.sol))
# print(f"step size: {sol_U_rk45.step_size:.6f}")
# print(U_dt)
# print(len(sol_U_rk45.t))
print(f"RK45 solver finished.")
# print(f"Time taken: {rk45_time:.4f} seconds")
# print(f"Number of steps: {len(sol_U_rk45.t)}")
# print(f"step size: {U_dt:.6f} ns")
print(f"Propagator U(T) computed. Starting stroboscopic evolution for {tUsteps} periods.")

# Reconstruct U from custom RK45
y_final_rk45 = sol_U_rk45.y[:, -1]
U_flat_rk45 = y_final_rk45[:D_sq] + 1j * y_final_rk45[D_sq:]
U_rk45 = U_flat_rk45.reshape((D_total, D_total), order='F')
solver_results['RK45'] = {'time': rk45_time, 'U': U_rk45, 'steps': len(sol_U_rk45.t)}

# --- Process Results (Iterative Evolution) ---
print("Calculating expectation values...")
njt = np.zeros((len(tlist), 2 * N))
psi_t = psi0_vec # Start with the initial state vector

for i, t in enumerate(tlist):
    # Measure expectation values
    for j in range(2 * N):
        op_j = number_ops[j]
        # <psi|O|psi> = (psi_conj.T) @ O @ psi
        exp_val = np.conj(psi_t) @ (op_j @ psi_t)
        njt[i, j] = exp_val.real
    
    # Evolve state for the next step
    psi_t = U_rk45 @ psi_t

total_time = time.time() - start_time
print(f"\nTotal ODE solver time: {total_time:.2f} seconds")


# --- Create Plot ---
fig, ax = plt.subplots(figsize=(12, 6))

for i in range(njt.shape[1]):
    if i % 2 == 0:  # Transmon site
        label_base = f'Transmon_{i//2 + 1}'
        plot_label = f'<n> Transmon {i//2 + 1}'
    else:  # Resonator site
        label_base = f'Resonator_{i//2 + 1}'
        plot_label = f'<n> Resonator {i//2 + 1}'

    expect_values = njt[:,i]
    ax.plot(tlist, expect_values, label=plot_label)

# --- Configure and Show Plot ---
ax.set_xlabel("Time (ns)")
ax.set_ylabel("Population")
ax.legend(loc='upper left')
ax.grid(True)
plt.show()



TEST 1: SciPy's Default RK45 Solver (Adaptive)
Starting RK45 solver...
RK45 solver finished.
Propagator U(T) computed. Starting stroboscopic evolution for 28693 periods.


C:\Users\Gonzalo\AppData\Local\Temp\ipykernel_16652\3702556192.py:24: UserWarning: The following arguments have no effect for a chosen solver: `vectorised`.
  sol_U_rk45 = RK45(


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

### 2. Custom RK45

In [ ]:


# ============================================
# Test 2: Custom RK4 with adaptive Step Size
# ============================================
print("\n" + "="*60)
print("TEST 2: Custom RK4 Solver (Fixed Step Size)")
print("="*60)

start_time = time.time()
print(f"Starting CustomRK4 solver (step size: {U_dt:.6f} ns)...")
sol_U_custom = solve_ivp(
    propagator_ode_real,
    t_span=[tlist_one_period[0], tlist_one_period[-1]],
    y0=y0_real,
    args=ode_args,
    method=CustomRK4,
    max_step=U_dt,
    dense_output=False
)
custom_time = time.time() - start_time
print(f"CustomRK4 solver finished.")
print(f"Time taken: {custom_time:.4f} seconds")
print(f"Number of steps: {len(sol_U_custom.t)}")

# Reconstruct U from CustomRK4
y_final_custom = sol_U_custom.y[:, -1]
U_flat_custom = y_final_custom[:D_sq] + 1j * y_final_custom[D_sq:]
U_custom = U_flat_custom.reshape((D_total, D_total), order='F')
solver_results['CustomRK4'] = {'time': custom_time, 'U': U_custom, 'steps': len(sol_U_custom.t)}

# --- Process Results (Iterative Evolution) ---
print(f"\nUsing custom RK45 propagator for stroboscopic evolution.")
print("Calculating expectation values...")
njt = np.zeros((len(tlist), 2 * N))
psi_t = psi0_vec # Start with the initial state vector

for i, t in enumerate(tlist):
    # Measure expectation values
    for j in range(2 * N):
        op_j = number_ops[j]
        # <psi|O|psi> = (psi_conj.T) @ O @ psi
        exp_val = np.conj(psi_t) @ (op_j @ psi_t)
        njt[i, j] = exp_val.real
    
    # Evolve state for the next step
    psi_t = U_custom @ psi_t

total_time = time.time() - start_time
print(f"\nTotal ODE solver time: {total_time:.2f} seconds")


# --- Create Plot ---
fig, ax = plt.subplots(figsize=(12, 6))

for i in range(njt.shape[1]):
    if i % 2 == 0:  # Transmon site
        label_base = f'Transmon_{i//2 + 1}'
        plot_label = f'<n> Transmon {i//2 + 1}'
    else:  # Resonator site
        label_base = f'Resonator_{i//2 + 1}'
        plot_label = f'<n> Resonator {i//2 + 1}'

    expect_values = njt[:,i]
    ax.plot(tlist, expect_values, label=plot_label)

# --- Configure and Show Plot ---
ax.set_xlabel("Time (ns)")
ax.set_ylabel("Population")
ax.legend(loc='upper left')
ax.grid(True)
plt.show()



TEST 2: Custom RK4 Solver (Fixed Step Size)
Starting CustomRK4 solver (step size: 0.000084 ns)...


KeyboardInterrupt: 

In [ ]:

# # ============================================
# # Comparison
# # ============================================
# print("\n" + "="*60)
# print("SOLVER COMPARISON")
# print("="*60)
# print(f"{'Solver':<15} {'Time (s)':<12} {'Steps':<8} {'Speedup':<10}")
# print("-" * 60)

# base_time = solver_results['RK45']['time']
# for solver_name in ['RK45', 'CustomRK4']:
#     result = solver_results[solver_name]
#     speedup = base_time / result['time']
#     print(f"{solver_name:<15} {result['time']:<12.4f} {result['steps']:<8} {speedup:>8.2f}x")

# ============================================
# Verify Results
# ============================================
# print("\n" + "="*60)
# print("SOLUTION COMPARISON")
# print("="*60)

# # Compare propagators (Frobenius norm of difference)
# U_diff = solver_results['RK45']['U'] - solver_results['CustomRK4']['U']
# frobenius_diff = np.linalg.norm(U_diff, 'fro')
# print(f"Frobenius norm of (RK45 - U_CustomRK4): {frobenius_diff:.2e}")

# # Use scipy for the rest of the simulation (more accurate)
# U = solver_results['RK45']['U']
# print(f"\nUsing RK45 propagator for stroboscopic evolution.")


### Look at the results